In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [30]:
dem_df = pd.read_csv('../data/post_dem_candidates_with_tweet_topics.csv', encoding="ISO-8859-1").dropna(subset=['primary_pctg'])
rep_df = pd.read_csv('../data/post_rep_candidates_with_tweet_topics.csv', encoding="ISO-8859-1").dropna(subset=['primary_pctg'])
tweets_with_topics = pd.read_csv('../data/tweet_data/tweets_with_topics.csv', index_col=False)
# sa_all_tweets = pd.read_csv('../data/tweet_data/sa_all_tweets_post.csv', index_col=False)

all_tweets_cleaned = pd.read_csv('../data/tweet_data/all_tweets_cleaned.csv', index_col=False)


In [39]:
tweets_with_topics = tweets_with_topics.merge(all_tweets_cleaned[['tweet_idx', 'twitter_handle']], how='left', on=['tweet_idx'])
                                              
                                              
                                              

In [34]:
print(len(all_tweets_cleaned))
print(len(tweets_with_topics))

557549
557549


In [ ]:
dem_candidates = dem_df.twitter_handle.unique()
dem_candidates2 = dem_df.twitter_handle2.unique()

party = []
for i, r in tweets_with_topics.iterrows():
    if i % 20000 == 0:
        print('{} / {}'.format(i, len(tweets_with_topics)))
    if r['twitter_handle'] in dem_candidates or r['twitter_handle'] in dem_candidates2:
        party.append(0)
    else:
        party.append(1)
tweets_with_topics['party'] = party

0 / 557549
20000 / 557549
40000 / 557549
60000 / 557549
80000 / 557549
100000 / 557549
120000 / 557549
140000 / 557549
160000 / 557549
180000 / 557549


In [4]:
topics = ['gun_control', 'health_care',
       'abortion', 'immigration', 'education', 'jobs'
          , 'environment','women', 'lgbt', 'freedom', 'trump']
topic_cols = ['{}_tweet_count'.format(t) for t in topics]

In [5]:
print("dem candidates w/ no classified topics: ", len(dem_df[(dem_df['gun_control_tweet_count'] == 0) & 
          (dem_df['health_care_tweet_count'] == 0) &
          (dem_df['abortion_tweet_count'] == 0) &
          (dem_df['immigration_tweet_count'] == 0) &
          (dem_df['education_tweet_count'] == 0) &
          (dem_df['jobs_tweet_count'] == 0) &
          (dem_df['environment_tweet_count'] == 0) &
          (dem_df['women_tweet_count'] == 0) &
          (dem_df['lgbt_tweet_count'] == 0) &
          (dem_df['freedom_tweet_count'] == 0) & (dem_df['trump_tweet_count'] == 0)]) / len(dem_df))

print("rep candidates w/ no classified topics: ", len(rep_df[(rep_df['gun_control_tweet_count'] == 0) & 
          (rep_df['health_care_tweet_count'] == 0) &
          (rep_df['abortion_tweet_count'] == 0) &
          (rep_df['immigration_tweet_count'] == 0) &
          (rep_df['education_tweet_count'] == 0) &
          (rep_df['jobs_tweet_count'] == 0) &
          (rep_df['environment_tweet_count'] == 0) &
          (rep_df['women_tweet_count'] == 0) &
          (rep_df['lgbt_tweet_count'] == 0) &
          (rep_df['freedom_tweet_count'] == 0) & (rep_df['trump_tweet_count'] == 0)]) / len(rep_df))

dem candidates w/ no classified topics:  0.22685788787483702
rep candidates w/ no classified topics:  0.4287769784172662


In [27]:


def get_topic_df(candidate_df, party):
    max_count_col = []
    median_count_col = []
    avg_count_col = []
    total_classified_col = []
    
    for t in topics:
        topic_count = len(tweets_with_topics[(tweets_with_topics[t] == 1) & (tweets_with_topics['party'] == party)])
        max_count = candidate_df["{}_tweet_count".format(t)].max()
        avg_count = candidate_df["{}_tweet_count".format(t)].mean()
        max_count_col.append(max_count)
        avg_count_col.append(round(avg_count, 2))
        total_classified_col.append(topic_count)
    df = pd.DataFrame(np.transpose([topics,avg_count_col, max_count_col, total_classified_col])
             , columns=['topic', 'avg_count', 'max_count', 'total_classified'])
    df['max_count'] = df['max_count'].astype('int32')
    df['total_classified'] = df['total_classified'].astype('int32')
    return df
    
dem_topic_df = get_topic_df(dem_df, 0)
rep_topic_df = get_topic_df(rep_df, 1)


In [28]:
dem_topic_df.sort_values(by=['max_count'], ascending=False)

,topic,avg_count,max_count,total_classified
6,environment,7.65,1026,6290
10,trump,28.24,554,35400
0,gun_control,15.01,352,14541
1,health_care,23.49,343,20787
3,immigration,7.04,192,9718
4,education,9.55,192,9030
5,jobs,13.25,157,14195
2,abortion,1.93,74,2879
8,lgbt,2.39,64,1900
7,women,2.03,59,1705


In [29]:
rep_topic_df.sort_values(by=['max_count'], ascending=False)

,topic,avg_count,max_count,total_classified
10,trump,19.53,1015,0
1,health_care,3.77,473,0
3,immigration,6.15,324,0
9,freedom,3.48,276,0
0,gun_control,4.23,254,0
5,jobs,5.74,200,0
4,education,2.47,83,0
2,abortion,1.96,75,0
6,environment,0.56,33,0
7,women,0.21,10,0
